In [12]:
import pandas as pd
import numpy as np
twitter = pd.read_csv("labeled_tweet_table.csv")
twitter

,Tweet,Name,Screen Name,Description,Lang,img_path,Race,Under 21
0,"YKAR, a futuristic sans serif font by @Emmeran...",Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
1,@MBonvoyAssist Who can I contact about the ver...,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
2,@SSlnes I’d like to win!,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
3,@LuckyDucksNFT @Rydog,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
4,Now I'm heading to B1000th Floor! #quickrogue,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NaN,profile pics/60147.jpeg,4.0,0
...,...,...,...,...,...,...,...,...
313468,Preach it 🙌🙌 https://t.co/8xPvSZpiLb,✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0
313469,Wut? https://t.co/FxR7WlXhDz,✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0
313470,"Im not trying to be rude, im just saying that ...",✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0
313471,"Sir, I'd like to know why you're being so rude...",✿♥☆✧աaʄʄʟɛ զʊɛɛռ✧☆♥✿,landry_rosalee,other acount- @furious_waffles and @i_cant_wor...,NaN,profile pics/64232.jpeg,NaN,0


In [13]:
import re
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

def labelFilter(label):
  return int(label)

twitter=twitter[["Tweet","Race"]]
#twitter["Tweet"]=twitter["Tweet"].apply(preprocess)
twitter = twitter.dropna()
twitter["Race"] = twitter["Race"].apply(labelFilter)
#twitter["Tweet"]
twitter

,Tweet,Race
0,"YKAR, a futuristic sans serif font by @Emmeran...",4
1,@MBonvoyAssist Who can I contact about the ver...,4
2,@SSlnes I’d like to win!,4
3,@LuckyDucksNFT @Rydog,4
4,Now I'm heading to B1000th Floor! #quickrogue,4
...,...,...
313398,I'm raising money for Help Feed the Homeless. ...,4
313399,I'm raising money for Help Feed the Homeless. ...,4
313400,I'm raising money for Help Feed the Homeless. ...,4
313401,I'm raising money for Help Feed the Homeless. ...,4


In [14]:
#!pip install tensorflow_text==2.5.0.
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import tensorflow_text as text

In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

sentences = ["I love you", "I hate you"]
preprocessed_text = bert_preprocess(sentences)
bert_encoder(preprocessed_text)["pooled_output"]

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-9.1058314e-01, -3.5070512e-01, -3.2318389e-01, ...,
        -4.5119785e-05, -7.1246189e-01,  9.1118807e-01],
       [-8.1203640e-01, -2.2017553e-01,  3.5842079e-01, ...,
         4.2030039e-01, -5.8162069e-01,  8.2115525e-01]], dtype=float32)>

In [16]:
data, labels = twitter["Tweet"], twitter["Race"]
data.shape, labels.shape

((310584,), (310584,))

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=.2)

In [9]:
text_input = keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

dense = keras.layers.Dense(6, activation='softmax')(outputs["pooled_output"])

model = keras.Model(inputs=[text_input], outputs=[dense])
model.summary()
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'default': (None, 7 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
5603/7765 [====================>.........] - ETA: 4:12 - loss: 0.8130 - accuracy: 0.7811

KeyboardInterrupt: 

In [19]:
from sklearn.model_selection import KFold

acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    text_input = keras.layers.Input(shape=(), dtype=tf.string)
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    dense = keras.layers.Dense(6, activation='softmax')(outputs["pooled_output"])

    model = keras.Model(inputs=[text_input], outputs=[dense])
    model.summary()
    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=50,
              epochs=5)

  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number
    fold_no = fold_no + 1


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'input_type_ids': ( 0           input_3[0][0]                    
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      {'default': (None, 7 109482241   keras_layer_2[1][0]              
                                                                 keras_layer_2[1][1]              
                                                                 keras_layer_2[1][2]              
____________________________________________________________________________________________

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = np.argmax(model.predict(x_test), axis=1)
accuracy_score(y_pred, y_test)